# Encoder Decoder 

### Imports

In [1]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.utils import shuffle

### Importing Data

In [31]:
#df = pd.read_csv('ASL_English.csv')
df = pd.read_csv('..\Data\Dataset_EnglishToGloss.csv')
df.head()

,English,ASL Gloss
0,I love to cook hamburgers on the grill.,G-R-I-L-L HAMBURGER COOK I LOVE.
1,I like to ice skate on our pond.,OUR P-O-N-D ICE SKATE I LIKE.
2,I like to be active and not sit and watch TV a...,ALL DAY I LIKE ACTIVE NOT SIT WATCH #TV.
3,Are you prepared for hurricane season?,HURRICANE S-E-A-S-O-N YOU READY PREPARE?
4,The dinner party was awkward because most of u...,DINNER PARTY AWKWARD WHY? MOST US NOT KNOW EA...


In [32]:
df.shape

(1428, 2)

### Preprocessing

Note:
- Replace the numbers/digits
- Check regarding Finger spellings
- Check if it is required to add start and end tokens to target sequences

In [33]:
replacements = {'1': " one ", '2':" two ", '3':" three ", '4':" four ", '5':" five ", '6':" six ", '7':" seven ", '8':" eight ", '9':" nine ", '0':" zero "}
df['English'] = df['English'].apply(lambda x: re.sub('(\d)', lambda m: replacements[m.group()], x))
df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: re.sub('(\d)', lambda m: replacements[m.group()], x))

# Remove extra spaces
df['English'] = df['English'].apply(lambda x: x.strip())
df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: x.strip())

# Lowercase all characters
df['English'] = df['English'].apply(lambda x: x.lower())
df['ASL Gloss'] = df['ASL Gloss'].apply (lambda x: x.lower())

# Removing double spaces
df['English'] = df['English'].apply(lambda x: x.replace('  ', ' '))
df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: x.replace('  ', ' '))

# Remove quotes # Might not need this
# df['English'] = df['English'].apply(lambda x: re.sub (r"'", '', x))
# df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: re.sub (r"'", '', x))

# Remove all special character
df['English'] = df['English'].apply(lambda x: ''.join (ch for ch in x if ch not in set(string.punctuation)))
df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: ''.join (ch for ch in x if ch not in set(string.punctuation)))

# Check if dataset has numbers
#print(df['English'].str.contains(r'\d').any())
#print(df['ASL Gloss'].str.contains(r'\d').any())

# Add tokens to target sequence
df['English'] = df['English'].apply(lambda x : 'START_ ' + x + ' _END')

In [34]:
df.head(10)

,English,ASL Gloss
0,START_ i love to cook hamburgers on the grill ...,grill hamburger cook i love
1,START_ i like to ice skate on our pond _END,our pond ice skate i like
2,START_ i like to be active and not sit and wat...,all day i like active not sit watch tv
3,START_ are you prepared for hurricane season _END,hurricane season you ready prepare
4,START_ the dinner party was awkward because mo...,dinner party awkward why most us not know each...
5,START_ my favorite author is me _END,my favorite author who me
6,START_ i love anything that has chocolate in i...,i love anything have chocolate inside
7,START_ in one nine four seven rhulin thomas be...,year one nine four seven rhulin thomas he firs...
8,START_ sean berdy is a deaf actor well known f...,sean berdy himself deaf actor famous why he in...
9,START_ did you see the movie about the life of...,you finish see movie about deaf woman herself ...


In [35]:
df.tail(10)

,English,ASL Gloss
1418,START_ the cow is brown _END,cow color what brown
1419,START_ the deer is tan _END,deer color what tan
1420,START_ the horse is white _END,horse color what white
1421,START_ the cat is black _END,cat color what black
1422,START_ the mouse is gray _END,mouse color what gray
1423,START_ the lion is gold _END,lion color what gold
1424,START_ the snake is silver _END,snake color what silver
1425,START_ i love whales _END,whales i love
1426,START_ i like frogs _END,frogs i like
1427,START_ juneteenth is tuesday june one nine _END,juneteenth when tuesday june one nine


In [36]:
# df['ASL Gloss'].str.len().sort_values(ascending=False).head()
# df['English'].str.len().sort_values(ascending=False).head()

In [37]:
# Vocabulary of English
all_eng_words = set()
for eng in df['English']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of ASL 
all_ASL_words = set()
for asl in df['ASL Gloss']:
    for word in asl.split():
        if word not in all_ASL_words:
            all_ASL_words.add(word)

In [38]:
# Max Length of source sequence
lenght_list=[]
for l in df ['English']:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
print("Max length target: ", max_length_tar)

Max length target:  72


In [39]:
# Max Length of target sequence
lenght_list=[]
for l in df ['ASL Gloss']:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
print("Max length sorce: ", max_length_src)

Max length sorce:  40


In [40]:
input_words = sorted(list(all_ASL_words))
target_words = sorted(list(all_eng_words))

# Calculate Vocab size for both source and target
num_encoder_tokens = len(all_ASL_words) + 1
num_decoder_tokens = len(all_eng_words) + 1

num_encoder_tokens, num_decoder_tokens

(1980, 2033)

In [41]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

2034

In [42]:
# Create word to token dictionary for both source and target
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

# Create token to word dictionary for both source and target
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [43]:
shuffle(df).head(10)

,English,ASL Gloss
519,START_ is your teacher deaf or hearing _END,your teacher deaf hearing which
220,START_ i want soup and salad for lunch _END,lunch soup salad i want
15,START_ i watched a boring movie last night and...,last night boring movie i watch fallasleep me
59,START_ i love watching sports every saturday _END,every saturday sports watch i love
726,START_ my parents are members of the indianapo...,indianapolis deaf club my parents members
1296,START_ the golf game is at five zero zero pm _END,time five zero zero pm golf game
1401,START_ i have seven shirts _END,seven shirts i have
613,START_ im not hungry ill just order soup _END,me not hungry soup order finish
454,START_ there was a famous protest at gallaudet...,back year one nine eight eight gallaudet unive...
1218,START_ i like trees i love pine trees _END,i like trees i love pine trees


Make a 90–10 train and test split and write a Python generator function to load the data in batches as follows:

In [44]:
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import tensorflow as tf

In [45]:
# Train - Test Split
X, y = df['ASL Gloss'], df['English']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((1285,), (143,))

Save the train and test dataframes for reproducing the results later, as they are shuffled.

In [46]:
X_train.to_pickle('Weights_ASL/X_train.pkl')
X_test.to_pickle('Weights_ASL/X_test.pkl')

In [47]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X.iloc[j:j+batch_size], y.iloc[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

Encoder - Decoder Model Architecture

In [48]:
latent_dim = 50

In [49]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [50]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

# Use a softmax to generate a probability distribution over the target vocabulary for each time step
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [51]:
# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

We train the network for 50 epochs with a batch size of 128.

In [84]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 200
# epochs = 10

In [85]:
model.fit(generate_batch(X_test, y_test, batch_size = batch_size),
            batch_size = batch_size,
            steps_per_epoch = train_samples//batch_size,
            epochs=epochs,
            validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
            validation_steps = val_samples//batch_size)

Epoch 1/200
10/10 [==============================] - 9s 894ms/step - loss: 4.3326 - acc: 0.1894 - val_loss: 4.9394 - val_acc: 0.1876
Epoch 2/200
10/10 [==============================] - 8s 826ms/step - loss: 4.3111 - acc: 0.1899 - val_loss: 4.9242 - val_acc: 0.1886
Epoch 3/200
10/10 [==============================] - 10s 1s/step - loss: 4.2902 - acc: 0.1909 - val_loss: 4.9021 - val_acc: 0.1895
Epoch 4/200
10/10 [==============================] - 10s 945ms/step - loss: 4.2690 - acc: 0.1923 - val_loss: 4.8808 - val_acc: 0.1905
Epoch 5/200
10/10 [==============================] - 12s 1s/step - loss: 4.2478 - acc: 0.1931 - val_loss: 4.8623 - val_acc: 0.1905
Epoch 6/200
10/10 [==============================] - 9s 907ms/step - loss: 4.2271 - acc: 0.1945 - val_loss: 4.8407 - val_acc: 0.1924
Epoch 7/200
10/10 [==============================] - 10s 979ms/step - loss: 4.2062 - acc: 0.1954 - val_loss: 4.8247 - val_acc: 0.1933
Epoch 8/200
10/10 [==============================] - 11s 1s/step - loss

Always remember to save the weights

In [86]:
model.save_weights('Weights_ASL/nmt_weights.h5')

Load the weights, if you close the application

In [55]:
model.load_weights('Weights_ASL/nmt_weights.h5')

Inference Setup

In [87]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

Finally, we generate the output sequence by invoking the above setup in a loop as follows

Decode sample sequeces

In [88]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

Evaluation on Train Dataset

In [89]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [101]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input ASL sentence:', X_train.iloc[k:k+1].values[0])
print('Actual English Translation:', y_train.iloc[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 50ms/step
Input ASL sentence: i like books
Actual English Translation:  i like books 
Predicted English Translation:  i like books 


In [102]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input ASL sentence:', X_train.iloc[k:k+1].values[0])
print('Actual English Translation:', y_train.iloc[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 75ms/step
Input ASL sentence: you like coffee tea which
Actual English Translation:  which do you like coffee or tea 
Predicted English Translation:  which do you like to go to go 


In [140]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input ASL sentence:', X_train.iloc[k:k+1].values[0])
print('Actual English Translation:', y_train.iloc[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 36ms/step
Input ASL sentence: i hate trees
Actual English Translation:  i hate trees 
Predicted English Translation:  i hate milk 


In [171]:
asl_sentence = []
true_eng_trans = []
pred_eng_trans = []

In [172]:
for i in range(10):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    asl_sentence.append(X_train.iloc[k:k+1].values[0])
    true_eng_trans.append(y_train.iloc[k:k+1].values[0][6:-4])
    pred_eng_trans.append(decoded_sentence[:-4])

1/1 [==============================] - 0s 36ms/step


In [173]:
for i in range(10):
    print('Input ASL sentence:', asl_sentence[i])
    print('Actual English Translation:', true_eng_trans[i])
    print('Predicted English Translation:', pred_eng_trans[i])
    print()

Input ASL sentence: my kids i teach help help other people why develops good character
Actual English Translation:  teaching my kids to help other people builds good character 
Predicted English Translation:  do you like bicycles 

Input ASL sentence: poem rhyme write can you
Actual English Translation:  can you write a poem that rhymes 
Predicted English Translation:  the wrestling is class is zero zero zero and basket

Input ASL sentence: yes i deaf
Actual English Translation:  yes im deaf 
Predicted English Translation:  i was a in 

Input ASL sentence: most famous deaf pilots who oneofthree cal rodgers twoofthree nellie zabel willhite threeofthree rhulin thomas
Actual English Translation:  three of the most famous deaf pilots are cal rodgers nellie zabel willhite and rhulin thomas 
Predicted English Translation:  hello brad nice to a get a slavery 

Input ASL sentence: butterfly color what purple
Actual English Translation:  the butterfly is purple 
Predicted English Translation:  